In [ ]:
!pip install timm
!pip install huggingface_hub
!huggingface-cli login 
 


In [ ]:
!pip install huggingface_hub


In [ ]:
from huggingface_hub import login

login(token="abc")



The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# from huggingface_hub import HfApi

# api = HfApi()
# api.upload_file(
#     path_or_fileobj="best.pt",  # your model weights
#     path_in_repo="pytorch_model.bin",  # standard filename
#     repo_id="your-username/earthguard-fastvit-classifier",
#     token="abc"  # or rely on saved config
# )


In [18]:
# 🧪 Codex Sandbox: EarthGuard Vision Model Refactor
# Project Goal: Improve inference speed, accuracy, and architectural robustness for object detection
# Context: Moving beyond YOLO if necessary. Model should be optimized for real-world litter detection from street/vehicle-mounted cameras.

# ✅ Setup
import torch
import torchvision
import timm  # PyTorch Image Models - for FastViT, ConvNeXt, etc.
from torchvision import transforms
from PIL import Image
import cv2
from huggingface_hub import HfApi

import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU count:", torch.cuda.device_count())
print("Current device:", torch.cuda.get_device_name(0))




CUDA available: True
GPU count: 1
Current device: NVIDIA GeForce RTX 3080 Ti


In [19]:
# 🧪 Codex Sandbox: EarthGuard Vision Model Refactor
# Project Goal: Improve inference speed, accuracy, and architectural robustness for object detection
# Context: Moving beyond YOLO if necessary. Model should be optimized for real-world litter detection from street/vehicle-mounted cameras.

# ✅ Setup
import torch
import torchvision
import timm  # PyTorch Image Models - for FastViT, ConvNeXt, etc.
from torchvision import transforms
from PIL import Image
import cv2
from huggingface_hub import HfApi

# 🧠 Debugging Check
print("CUDA available:", torch.cuda.is_available())
print("GPU count:", torch.cuda.device_count())
print("current device:", torch.cuda.get_device_name(0))

# ✅ Load image safely with existence check
def load_sample_image(path):
    try:
        image = Image.open(path).convert("RGB")
    except FileNotFoundError:
        print(f"Image file not found: {path}")
        return None
    transform = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0)

# 🚀 Load ConvNeXt model
model = timm.create_model("convnext_tiny.in12k_ft_in1k", pretrained=True)
model.eval()

# 🔁 Run inference
@torch.no_grad()
def run_inference(model, image_tensor):
    if image_tensor is None:
        return None
    return model(image_tensor)

# 🔂 Load and infer
image_path = "litter_survey_dataset/SampleImageToAnnotate/3B9D1465-EB6A-49E9-9B79-D3A19AB3B570_1_105_c.jpeg"
image_tensor = load_sample_image(image_path)
output = run_inference(model, image_tensor)
if output is not None:
    print("Output shape:", output.shape)

# 📤 Upload model to Hugging Face (optional step)
def upload_to_huggingface():
    api = HfApi()
    api.upload_file(
        path_or_fileobj="best.pt",
        path_in_repo="pytorch_model.bin",
        repo_id="yanglinfang/earthguard-fastvit-classifier"
    )
    print("Model uploaded to Hugging Face.")

# 🛠️ Future directions:
# - Plug into Detectron2 or MMDetection for box prediction
# - Add custom classification head for 'litter vs non-litter'
# - Export to ONNX or TorchScript for mobile/embedded inference


CUDA available: True
GPU count: 1
current device: NVIDIA GeForce RTX 3080 Ti
Output shape: torch.Size([1, 1000])


In [20]:
model = timm.create_model("convnext_tiny.in12k_ft_in1k", pretrained=True)
# Alternatives known to work
# "efficientnet_b0"
# "resnet50"
# "vit_base_patch16_224"


## Next Step: Train a Baseline YOLOv8 Detector

We'll start from a small pre-trained checkpoint and fine-tune on our custom dataset.

In [ ]:
from ultralytics import YOLO
import yaml
from pathlib import Path

# Path to dataset YAML
data_yaml = Path('train_dataset/train_dataset.yaml')

# Update this to the folder containing your dataset on Windows
dataset_root = Path(r'C:\path\to\train_dataset')  # change as needed

with open(data_yaml) as f:
    data_dict = yaml.safe_load(f)
data_dict['path'] = str(dataset_root)

# Initialize YOLOv8 with a small model. The weight file should exist locally or in the cache.
model = YOLO('yolov8n.pt')

# Fine-tune on our dataset for a few epochs
model.train(data=data_dict, epochs=5, imgsz=640)

# Evaluate on the validation set
metrics = model.val()
print(metrics)


### Windows Setup Notes

If you're running this notebook on Windows, set `dataset_root` in the previous cell to the path of your local dataset folder. The folder should contain `images/` and `labels/` subdirectories.

## Optional: Export to ONNX for Deployment

In [ ]:
# Export the trained model to ONNX format for use in other runtimes
model.export(format='onnx', dynamic=True)
